In [2]:
import pandas as pd
import panel as pn
import psycopg2
from sqlalchemy import create_engine
from bokeh.models import CheckboxGroup
from data.database import conectar_postgres

pn.extension()

# Função para pesquisar animais no PostgreSQL com base em um filtro
def pesquisar_animais(filtro_coluna, filtro_valor):
    conn = conectar_postgres()
    if conn:
        engine = create_engine('postgresql+psycopg2://', creator=lambda: conn)
        query = f"SELECT * FROM Animais WHERE {filtro_coluna} = '{filtro_valor}'"
        result_df = pd.read_sql_query(query, engine)
        conn.close()
        return result_df if result_df is not None else pd.DataFrame()
    return pd.DataFrame()

# Funções CRUD para Animais
def consultar_animais():
    conn = conectar_postgres()
    if conn:
        engine = create_engine('postgresql+psycopg2://', creator=lambda: conn)
        query = "SELECT * FROM Animais"
        result_df = pd.read_sql_query(query, engine)
        conn.close()
        return result_df if result_df is not None else pd.DataFrame()
    return pd.DataFrame()

def inserir_animal(nome_animal, raca, dono, permanente, saude_ao_registro, estado):
    conn = conectar_postgres()
    if conn:
        with conn.cursor() as cursor:
            # Se permanente estiver marcado, temporario é automaticamente desmarcado
            if permanente:
                cursor.execute(
                    "INSERT INTO Animais (nome_animal, raca, dono, permanente, saude_ao_registro, estado) VALUES (%s, %s, %s, %s, %s, %s)",
                    (nome_animal, raca, dono, permanente, saude_ao_registro, estado)
                )
            else:
                cursor.execute(
                    "INSERT INTO Animais (nome_animal, raca, dono, permanente, saude_ao_registro, estado) VALUES (%s, %s, %s, %s, %s, %s)",
                    (nome_animal, raca, dono, permanente, saude_ao_registro, estado)
                )
            conn.commit()
        conn.close()


# Widget para exibir animais
animais_output = pn.widgets.DataFrame()

nome_animal_input = pn.widgets.TextInput(name="Nome")
raca_input = pn.widgets.TextInput(name="Raça")
dono_input = pn.widgets.TextInput(name="Proprietário")
permanente_input = pn.widgets.Checkbox(name="Permanente")
temporario_input = pn.widgets.Checkbox(name="Temporário")
saude_ao_registro_input = pn.widgets.IntInput(name="Saúde ao Registro")
estado_input = pn.widgets.TextInput(name="Estado, ex: Ceará")
id_animal_input = pn.widgets.TextInput(name="ID do Animal (para atualização/exclusão)")
filtro_valor_input = pn.widgets.TextInput(name="Valor de Pesquisa")
atualizar_button = pn.widgets.Button(name="Atualizar")

# Função para atualizar um animal no banco de dados
def atualizar_animal(animal_id, **kwargs):
    conn = conectar_postgres()
    if conn:
        with conn.cursor() as cursor:
            set_clauses = ', '.join([f"{key} = %s" for key in kwargs.keys()])
            update_query = f"UPDATE Animais SET {set_clauses} WHERE id_animal = %s"
            cursor.execute(update_query, list(kwargs.values()) + [animal_id])
            
            conn.commit()
        conn.close()

# Função para atualizar os campos modificados do animal
def on_atualizar_animal_button_click(event):
    animal_id = int(id_animal_input.value)
    nome_animal = nome_animal_input.value
    raca = raca_input.value
    dono = dono_input.value
    permanente = permanente_input.value
    temporario = temporario_input.value
    saude_ao_registro = saude_ao_registro_input.value
    estado = estado_input.value
        
    # Atualizar os campos modificados
    if nome_animal:
        atualizar_animal(animal_id, nome_animal=nome_animal)
    if raca:
        atualizar_animal(animal_id, raca=raca)
    if dono:
        atualizar_animal(animal_id, dono=dono)
    if permanente:
        atualizar_animal(animal_id, permanente=permanente)
    if temporario:
        atualizar_animal(animal_id, permanente=False)
    if saude_ao_registro:
        atualizar_animal(animal_id, saude_ao_registro=saude_ao_registro)
    if estado:
        atualizar_animal(animal_id, estado=estado)

    on_exibir_animais_button_click(None)

# Botão de Inserir
inserir_button = pn.widgets.Button(name="Inserir")


# Função para inserir um novo animal
def on_inserir_button_click(event):
    nome_animal = nome_animal_input.value
    raca = raca_input.value
    dono = dono_input.value
    permanente = permanente_input.value
    temporario = temporario_input.value
    saude_ao_registro = saude_ao_registro_input.value
    estado = estado_input.value if estado_input.value != '' else None

    # Inserir o novo animal no banco de dados
    inserir_animal(nome_animal, raca, dono, permanente, saude_ao_registro, estado)

    # Limpar os campos de entrada
    nome_animal_input.value = ''
    raca_input.value = ''
    dono_input.value = ''
    permanente_input.value = False
    temporario_input.value = False
    saude_ao_registro_input.value = None 
    estado_input.value = ''

    # Atualizar a exibição dos animais
    on_exibir_animais_button_click(event)

def on_excluir_button_click(event):
    animal_id = int(id_animal_input.value)
    excluir_animal(animal_id)
    on_exibir_animais_button_click(event)


# Conectar função ao evento de clique do botão de Inserir
inserir_button.on_click(on_inserir_button_click)

# Botão de Excluir
excluir_button = pn.widgets.Button(name="Excluir")

def excluir_animal(animal_id):
    conn = conectar_postgres()
    if conn:
        try:
            # Remover registros relacionados na tabela "senhas_numeros"
            with conn.cursor() as cursor:
                cursor.execute("DELETE FROM senhas_numeros WHERE id_animal = %s", (animal_id,))
                conn.commit()

            # Excluir o animal
            with conn.cursor() as cursor:
                cursor.execute("DELETE FROM Animais WHERE id_animal = %s", (animal_id,))
                conn.commit()

        except psycopg2.Error as e:
            print(f"Erro ao excluir animal: {e}")
        finally:
            conn.close()

# Botão de Pesquisar
pesquisar_button = pn.widgets.Button(name="Pesquisar")

# Função para pesquisar animais
def on_pesquisar_button_click(event):
    filtro_coluna = filtro_coluna_dropdown.value
    filtro_valor = filtro_valor_input.value

    if filtro_coluna == "ID":
        # Tratando pesquisa por ID
        try:
            filtro_valor = int(filtro_valor)
        except ValueError:
            print("ID deve ser um número inteiro.")
            return

    result_df = pesquisar_animais(filtro_coluna, filtro_valor)
    if result_df is not None:
        animais_output.value = result_df


# Botão de Exibir/Ocultar Animais
exibir_button = pn.widgets.Button(name="Exibir Animais")


def on_exibir_animais_button_click(event):
    result_df = consultar_animais()
    if result_df is not None:
        animais_output.value = result_df
        animais_output.visible = not animais_output.visible

# Dropdown para seleção do campo de filtro
filtro_coluna_dropdown = pn.widgets.Select(options=["id_animal","nome_animal", "raca", "dono", "permanente","saude_ao_registro", "estado"], name="Filtrar por")

# Chamar a função para exibir os animais ao criar o layout
on_exibir_animais_button_click(None)

# Conectar funções aos eventos dos botões
atualizar_button.on_click(on_atualizar_animal_button_click)
excluir_button.on_click(on_excluir_button_click)
pesquisar_button.on_click(on_pesquisar_button_click)
exibir_button.on_click(on_exibir_animais_button_click)

# Montar layout
layout = pn.Column(
    "## CRUD de Animais",
    nome_animal_input,
    dono_input,
    pn.Row(permanente_input, temporario_input),
    estado_input,
    pn.Row(saude_ao_registro_input,"1-Saudável", "2-Cansado", "3-Impossibilitado"),
    inserir_button,
    "",
    pn.Row(id_animal_input, excluir_button,atualizar_button,),
    pn.Row(filtro_valor_input, filtro_coluna_dropdown, pesquisar_button),
    pn.Row(animais_output, exibir_button)
)   

# Chamar a função para exibir os animais ao criar o layout
on_exibir_animais_button_click(None)

# Exibir layout
layout


Column
    [0] Markdown(str)
    [1] TextInput(name='Nome')
    [2] TextInput(name='Proprietário')
    [3] Row
        [0] Checkbox(name='Permanente')
        [1] Checkbox(name='Temporário')
    [4] TextInput(name='Estado, ex: Ceará')
    [5] Row
        [0] IntInput(name='Saúde ao Registro')
        [1] Markdown(str)
        [2] Markdown(str)
        [3] Markdown(str)
    [6] Button(name='Inserir')
    [7] Markdown(str)
    [8] Row
        [0] TextInput(name='ID do Animal (...)
        [1] Button(name='Excluir')
        [2] Button(name='Atualizar')
    [9] Row
        [0] TextInput(name='Valor de Pesquisa')
        [1] Select(name='Filtrar por', options=['id_animal', ...], value='id_animal')
        [2] Button(name='Pesquisar')
    [10] Row
        [0] DataFrame(value=    id_animal  ...)
        [1] Button(name='Exibir Animais')